In [1]:
!pip install dash

   ---------------------------------------- 0.0/7.8 MB ? eta -:--:--
    --------------------------------------- 0.1/7.8 MB 4.3 MB/s eta 0:00:02
   -- ------------------------------------- 0.5/7.8 MB 5.6 MB/s eta 0:00:02
   --- ------------------------------------ 0.7/7.8 MB 5.5 MB/s eta 0:00:02
   ----- ---------------------------------- 1.0/7.8 MB 5.9 MB/s eta 0:00:02
   ------ --------------------------------- 1.2/7.8 MB 5.4 MB/s eta 0:00:02
   ------ --------------------------------- 1.2/7.8 MB 4.6 MB/s eta 0:00:02
   ------- -------------------------------- 1.5/7.8 MB 4.6 MB/s eta 0:00:02
   -------- ------------------------------- 1.6/7.8 MB 4.5 MB/s eta 0:00:02
   --------- ------------------------------ 1.8/7.8 MB 4.4 MB/s eta 0:00:02
   ---------- ----------------------------- 2.0/7.8 MB 4.4 MB/s eta 0:00:02
   ----------- ---------------------------- 2.2/7.8 MB 4.3 MB/s eta 0:00:02
   ------------ --------------------------- 2.4/7.8 MB 4.3 MB/s eta 0:00:02
   ------------- --

In [93]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import pandas as pd
import plotly.express as px

# Load and prepare the cleaned data
# df = pd.read_csv('C:/Users/naish/Downloads/Capstone Project/datasets/expense_data_1.csv')  # Adjust path as needed
# df_cleaned = df[['Date', 'Category', 'Note', 'INR', 'Income/Expense']].copy()
# df_cleaned.columns = ['Date', 'Category', 'Description', 'Amount', 'Type']
# df_cleaned['Date'] = pd.to_datetime(df_cleaned['Date'], errors='coerce')
# df_cleaned.dropna(subset=['Date', 'Category', 'Amount', 'Type'], inplace=True)
# df_cleaned['Amount'] = pd.to_numeric(df_cleaned['Amount'], errors='coerce')

# Initialize the Dash app
app = dash.Dash(__name__)

# Layout of the Dash app with added filters
app.layout = html.Div([
    html.H1("Personal Financial Dashboard"),

    # Dropdown for selecting time range
    dcc.Dropdown(
        id='time-range',
        options=[
            {'label': 'Daily', 'value': 'Daily'},
            {'label': 'Weekly', 'value': 'Weekly'},
            {'label': 'Monthly', 'value': 'Monthly'},
            {'label': 'Yearly', 'value': 'Yearly'},
            {'label': 'Custom Dates', 'value': 'Custom'}
        ],
        value='Monthly',
        clearable=False
    ),

    # Date picker for custom date range
    dcc.DatePickerRange(
        id='custom-date-picker',
        start_date=df_cleaned['Date'].min(),
        end_date=df_cleaned['Date'].max(),
        style={'display': 'none'}  # Initially hidden
    ),

    # Dropdown for selecting transaction type (Income or Expense)
    dcc.Dropdown(
        id='transaction-type',
        options=[
            {'label': 'All', 'value': 'All'},
            {'label': 'Income', 'value': 'Income'},
            {'label': 'Expense', 'value': 'Expense'}
        ],
        value='All',
        clearable=False
    ),

    # Graph to show transactions over time
    dcc.Graph(id='transactions-time-series'),

    # Pie chart for category-wise spending
    dcc.Graph(id='category-pie-chart'),

    # Bar chart for monthly spending
    dcc.Graph(id='monthly-spending-bar-chart')
])

# Callback to show or hide date picker based on dropdown selection
@app.callback(
    Output('custom-date-picker', 'style'),
    [Input('time-range', 'value')]
)
def show_date_picker(time_range):
    if time_range == 'Custom':
        return {'display': 'block'}
    else:
        return {'display': 'none'}

# Callback to update graphs based on date range and transaction type
@app.callback(
    [Output('transactions-time-series', 'figure'),
     Output('category-pie-chart', 'figure'),
     Output('monthly-spending-bar-chart', 'figure')],
    [Input('time-range', 'value'),
     Input('transaction-type', 'value'),
     Input('custom-date-picker', 'start_date'),
     Input('custom-date-picker', 'end_date')]
)
def update_graphs(time_range, transaction_type, start_date, end_date):
    # Filter data based on transaction type
    if transaction_type == 'All':
        filtered_df = df_cleaned.copy()
    else:
        filtered_df = df_cleaned[df_cleaned['Type'] == transaction_type]

    # Filter data based on selected date range
    if time_range == 'Custom':
        filtered_df = filtered_df[(filtered_df['Date'] >= pd.to_datetime(start_date)) & (filtered_df['Date'] <= pd.to_datetime(end_date))]
    elif time_range == 'Daily':
        filtered_df = filtered_df.groupby(filtered_df['Date'].dt.date).agg({'Amount': 'sum'}).reset_index()
        filtered_df.rename(columns={'Date': 'Date'}, inplace=True)
    elif time_range == 'Weekly':
        filtered_df = filtered_df.groupby(filtered_df['Date'].dt.to_period('W').apply(lambda r: r.start_time)).agg({'Amount': 'sum'}).reset_index()
    elif time_range == 'Monthly':
        filtered_df = filtered_df.groupby(filtered_df['Date'].dt.to_period('M').apply(lambda r: r.start_time)).agg({'Amount': 'sum'}).reset_index()
    elif time_range == 'Yearly':
        filtered_df = filtered_df.groupby(filtered_df['Date'].dt.to_period('Y').apply(lambda r: r.start_time)).agg({'Amount': 'sum'}).reset_index()

    # Ensure data is available for plotting
    if filtered_df.empty:
        return {}, {}, {}

    # Transactions over time
    time_series_fig = px.line(filtered_df, x='Date', y='Amount', title='Transactions Over Time')

    # Filter data for category pie chart based on date range and transaction type
    if time_range == 'Custom':
        pie_df = df_cleaned[(df_cleaned['Date'] >= pd.to_datetime(start_date)) & (df_cleaned['Date'] <= pd.to_datetime(end_date))]
    else:
        pie_df = df_cleaned[df_cleaned['Date'].isin(filtered_df['Date'])]

    if transaction_type != 'All':
        pie_df = pie_df[pie_df['Type'] == transaction_type]

    # Category-wise spending pie chart
    category_pie_chart = px.pie(pie_df, names='Category', values='Amount', title='Category-wise Distribution')

    # Monthly spending bar chart
    if time_range != 'Custom':
        monthly_spending_fig = px.bar(filtered_df, x='Date', y='Amount', title='Spending Over Selected Time Range')
    else:
        filtered_df['Month'] = filtered_df['Date'].dt.to_period('M').astype(str)
        monthly_spending_fig = px.bar(filtered_df, x='Month', y='Amount', title='Monthly Spending')

    return time_series_fig, category_pie_chart, monthly_spending_fig

# Run the Dash app
if __name__ == '__main__':
    app.run_server(debug=True)

C:\Users\naish\anaconda3\Lib\site-packages\_plotly_utils\basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result

C:\Users\naish\anaconda3\Lib\site-packages\_plotly_utils\basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result

C:\Users\naish\anaconda3\Lib\site-packages\_plotly_utils\basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



In [8]:
import sqlite3

In [76]:
conn=sqlite3.connect("categorised_transaction.db")

In [77]:
cur=conn.cursor()

In [92]:
cur.execute("select * from transactions;")
print(cur.fetchall())

[(None, '2022-01-05 03:26:21.000000', 'food', 'To kumara', 150, 'Expense', 'Not Important'), (None, '2022-01-06 11:27:47.000000', 'entertainment', 'museum ticket', 250, 'Expense', 'Not Important'), (None, '2022-01-07 12:19:50.000000', 'transportation', 'car wash', 400, 'Expense', 'Not Important'), (None, '2022-01-14 04:04:05.000000', 'others', 'notepad', 80, 'Expense', 'Not Important'), (None, '2022-01-15 11:00:40.000000', 'others', 'cup coffee', 150, 'Expense', 'Not Important'), (None, '2022-01-16 09:04:29.000000', 'food', 'chocolates', 100, 'Expense', 'Not Important'), (None, '2022-01-19 15:55:36.000000', 'others', 'Salary from dad', 5000, 'Income', 'Not Important'), (None, '2022-01-25 22:40:47.000000', 'food', 'Snacks', 250, 'Expense', 'Not Important'), (None, '2022-01-29 16:42:47.000000', 'food', 'Pizza', 301.75, 'Expense', 'Not Important'), (None, '2022-01-30 05:33:51.000000', 'others', 'To vishnu', 100, 'Expense', 'Not Important'), (None, '2022-02-01 07:52:30.000000', 'food', Non

In [90]:
cur.execute("select * from transactions;")
df=pd.DataFrame(cur.fetchall())
df.columns = ['transaction_id', 'Date', 'Category', 'Description', 'Amount', 'Type', 'Importance']
df.drop('transaction_id', axis=1, inplace=True)
df.dropna(axis=0, inplace=True)
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
print(df)

                   Date        Category    Description  Amount     Type  \
0   2022-01-05 03:26:21            food      To kumara   150.0  Expense   
1   2022-01-06 11:27:47   entertainment  museum ticket   250.0  Expense   
2   2022-01-07 12:19:50  transportation       car wash   400.0  Expense   
3   2022-01-14 04:04:05          others        notepad    80.0  Expense   
4   2022-01-15 11:00:40          others     cup coffee   150.0  Expense   
..                  ...             ...            ...     ...      ...   
536 2024-10-20 01:27:35        shopping        clothes   600.0  Expense   
537 2024-10-28 01:53:20          others         others  3490.0  Expense   
538 2024-10-31 20:31:50   entertainment  museum ticket   350.0  Expense   
539 2024-11-02 20:38:18          others     From vicky   300.0   Income   
540 2024-11-02 20:44:02          others       bookmark    40.0  Expense   

        Importance  
0    Not Important  
1    Not Important  
2    Not Important  
3    Not Import

In [91]:
df_cleaned=df

In [71]:
cur.close()
conn.close()